# Categorical Data

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

## Background and Motivation

In [2]:
values= pd.Series(['apple', 'orange', 'apple', 'apple']*2)
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [3]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [4]:
pd.value_counts(values)

C:\Users\Najee\AppData\Local\Temp\ipykernel_15712\3297668723.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(values)


apple     6
orange    2
Name: count, dtype: int64

In [5]:
values = pd.Series([0,1,0,0]*2)
dim = pd.Series(['apple', 'orange'])

In [6]:
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [7]:
dim

0     apple
1    orange
dtype: object

In [8]:
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

- This representation as integers is called the categorical or dictionary-encoded representation.
- The array of distinct values can be called the categories, dictonary, or levels of the data.
- The integer values that reference the categories are called the category codes or simply codes.

- Categorical representation can yield significant performance improvements.
- Transformation on categories can be performed without touching the codes.
- Examples:
    - Renaming Categories
    - Appending a new cayegory without changing the order or position of the existing categories

## Categorical Extension Type in Pandas

- pandas has a special `Categorical` extension type of holding data that uses the integer-based categorical represetaton or encoding.
- This is a popular data compression technique.
    - significant faster performance
    - lower memory use, especiall for string data

In [9]:
fruits = ['apple', 'orange', 'apple', 'apple']*2

In [10]:
N = len(fruits)

In [11]:
rng = np.random.default_rng(seed=12345)

In [14]:
df = pd.DataFrame({'fruit': fruits,
                    'basket_id': np.arange(N),
                    'count': rng.integers(3, 15, size=N),
                    'weight': rng.uniform(0, 4, size=N)}, 
                    columns=['basket_id', 'fruit', 'count', 'weight'])

In [15]:
df

,basket_id,fruit,count,weight
0,0,apple,11,1.564438
1,1,orange,5,1.331256
2,2,apple,12,2.393235
3,3,apple,6,0.746937
4,4,apple,5,2.691024
5,5,orange,12,3.767211
6,6,apple,10,0.992983
7,7,apple,11,3.795525


In [ ]:
df['fruit']

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: object

In [17]:
fruit_cat = df['fruit'].astype('category')

In [18]:
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

- The values for `fruit_cat` are now an instance of `pandas.Categorical`, which we can access via the `.array` attribute.

In [19]:
c = fruit_cat.array

In [20]:
type(c)

pandas.core.arrays.categorical.Categorical

In [21]:
c

['apple', 'orange', 'apple', 'apple', 'apple', 'orange', 'apple', 'apple']
Categories (2, object): ['apple', 'orange']

In [22]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [23]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

> A useful trick to get a mapping between codes and categories

In [24]:
dict(enumerate(c.categories))

{0: 'apple', 1: 'orange'}

In [26]:
for cp in enumerate(c.categories):
    print(f"type of cp: {type(cp)}. and cp = {cp}")

type of cp: <class 'tuple'>. and cp = (0, 'apple')
type of cp: <class 'tuple'>. and cp = (1, 'orange')


In [31]:
df['fruit'] = df['fruit'].astype('category')
df['fruit']

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [32]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
my_categories

['foo', 'bar', 'baz', 'foo', 'bar']
Categories (3, object): ['bar', 'baz', 'foo']

In [33]:
categories = ['foo', 'bar', 'baz']
codes = [0,1,2,0,0,1]
my_cats_2 = pd.Categorical.from_codes(codes, categories=categories)
my_cats_2

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz']

In [35]:
ordered_cat = pd.Categorical.from_codes(codes, categories, ordered=True)
ordered_cat

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

In [36]:
my_cats_2.as_ordered()

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

## Computations with Categoricals

- Group by functions perform better when working with categoricals

In [37]:
draws = rng.standard_normal(1000)
draws[:5]

array([-0.75938718,  0.90219827, -0.46695317, -0.06068952,  0.78884434])

Lets compute some quatile binning of this data and extract sine statistics:

In [38]:
bins = pd.qcut(draws, 4)
bins

[(-3.121, -0.673], (0.693, 3.211], (-0.673, 0.0198], (-0.673, 0.0198], (0.693, 3.211], ..., (0.693, 3.211], (0.0198, 0.693], (0.0198, 0.693], (0.693, 3.211], (0.693, 3.211]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.121, -0.673] < (-0.673, 0.0198] < (0.0198, 0.693] < (0.693, 3.211]]

In [39]:
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3','Q4'])
bins

['Q1', 'Q4', 'Q2', 'Q2', 'Q4', ..., 'Q4', 'Q3', 'Q3', 'Q4', 'Q4']
Length: 1000
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [40]:
bins.codes[:10]

array([0, 3, 1, 1, 3, 0, 2, 3, 3, 1], dtype=int8)

- The labeled bins categorical does not contain information about the bin edges in the data, so we can use `goupby` to extract some summary statistics:

In [41]:
bins = pd.Series(bins, name='quartile')

In [42]:
results = (pd.Series(draws)
            .groupby(bins)
            .agg(['count', 'min', 'max'])
            .reset_index())

C:\Users\Najee\AppData\Local\Temp\ipykernel_15712\2784864137.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  results = (pd.Series(draws)


In [43]:
results

,quartile,count,min,max
0,Q1,250,-3.119609,-0.673305
1,Q2,250,-0.672703,0.019655
2,Q3,250,0.019958,0.692732
3,Q4,250,0.695674,3.211418


- The `quartile` column retains the original categoricals information, including ordering from bins

In [44]:
results['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

## Better performance with categoricals

In [45]:
N = 10_000_000
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))

In [46]:
categories = labels.astype('category')

In [47]:
labels.memory_usage(deep=True)

600000128

In [48]:
categories.memory_usage(deep=True)

10000540

In [49]:
%timeit _ = labels.astype('category')

586 ms ± 68.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [50]:
%timeit labels.value_counts()

503 ms ± 15.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [51]:
%timeit categories.value_counts()

60.1 ms ± 7.31 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Categorical Methods

In [52]:
s = pd.Series(list('abcd')*2)

In [53]:
cat_s = s.astype('category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

- Special accessor attribute `cat` provides access to categorical methods

In [54]:
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [55]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [56]:
actual_categories = list('abcde')
cat_s2 = cat_s.cat.set_categories(actual_categories)

In [57]:
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): ['a', 'b', 'c', 'd', 'e']

In [58]:
cat_s.value_counts()

a    2
b    2
c    2
d    2
Name: count, dtype: int64

In [59]:
cat_s2.value_counts()

a    2
b    2
c    2
d    2
e    0
Name: count, dtype: int64

In [60]:
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [61]:
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): ['a', 'b']